<a href="https://colab.research.google.com/github/theamrzaki/text_summurization_abstractive_methods/blob/master/Implementation%20B%20(Pointer%20Generator%20seq2seq%20network)/zaksum_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Google Drive

In [2]:
#https://stackoverflow.com/questions/47744131/colaboratory-can-i-access-to-my-google-drive-folder-and-file

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110845 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

### Parser

In [0]:
article=[]
reference=[]
summary =[]

import xml.etree.ElementTree
e = xml.etree.ElementTree.parse("drive/Colab Notebooks/Model 4 generator/result_pointer_4_1_2019_9_39pm.xml").getroot()

for atype in e.findall('example/article'):
    article.append(atype.text)
    
for atype in e.findall('example/reference'):
     reference.append(atype.text)
    
for atype in e.findall('example/summary'):
     summary.append(atype.text)

### zaksum

In [11]:
!pip install sumeval
!python -m spacy download en

    100% |████████████████████████████████| 51kB 2.0MB/s 
  Running setup.py bdist_wheel for sumeval ... - \ done
  Stored in directory: /root/.cache/pip/wheels/f1/7c/18/177331e7a5154401ab5fec8f3c6d2062508c38d832aaa3d5e2
  Running setup.py bdist_wheel for sacrebleu ... - done
  Stored in directory: /root/.cache/pip/wheels/ea/0a/7d/ddcbdcd15a04b72de1b3f78e7e754aab415aff81c423376385
Successfully built sumeval sacrebleu

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
#https://github.com/chakki-works/sumeval
#https://github.com/Tian312/awesome-text-summarization

from sumeval.metrics.rouge import RougeCalculator
from sumeval.metrics.bleu import BLEUCalculator

def eval_rouges(refrence_summary,model_summary):
    rouge = RougeCalculator(stopwords=True, lang="en")

    rouge_1 = rouge.rouge_n(
                summary=model_summary,
                references=refrence_summary,
                n=1)

    rouge_2 = rouge.rouge_n(
                summary=model_summary,
                references=[refrence_summary],
                n=2)
    
    rouge_l = rouge.rouge_l(
                summary=model_summary,
                references=[refrence_summary])
    
    # You need spaCy to calculate ROUGE-BE
    
    rouge_be = rouge.rouge_be(
                summary=model_summary,
                references=[refrence_summary])

    bleu = BLEUCalculator()
    bleu_score = bleu.bleu( summary=model_summary,
                        references=[refrence_summary])
    
    return rouge_1, rouge_2,rouge_l,rouge_be,bleu_score

In [0]:
#https://pymotw.com/2/xml/etree/ElementTree/create.html

bleu_arr = []
rouge_1_arr  = []
rouge_2_arr  = []
rouge_L_arr  = []
rouge_be_arr = []

from xml.etree import ElementTree
from xml.dom import minidom
from functools import reduce

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")
  
from xml.etree.ElementTree import Element, SubElement, Comment

top = Element('ZakSum')

def zaksum(article,reference,summary_array,default_path):
  comment = Comment('Generated by Amr Zaki')
  top.append(comment)

  i=0
  for summ in summary_array:
    example = SubElement(top, 'example')
    article_element   = SubElement(example, 'article')
    article_element.text = article[i]

    reference_element = SubElement(example, 'reference')
    reference_element.text = reference[i]

    summary_element   = SubElement(example, 'summary')
    summary_element.text = summ

    rouge_1, rouge_2,rouge_L,rouge_be,bleu_score = eval_rouges(reference[i],summ )

    eval_element = SubElement(example, 'eval')
    bleu_score_element = SubElement(eval_element,'BLEU', {'score':str(bleu_score)})
    ROUGE_1_element  = SubElement(eval_element, 'ROUGE_1' , {'score':str(rouge_1)})
    ROUGE_2_element  = SubElement(eval_element, 'ROUGE_2' , {'score':str(rouge_2)})
    ROUGE_L_element  = SubElement(eval_element, 'ROUGE_l' , {'score':str(rouge_L)})
    ROUGE_be_element  = SubElement(eval_element,'ROUGE_be', {'score':str(rouge_be)})

    bleu_arr.append(bleu_score) 
    rouge_1_arr.append(rouge_1) 
    rouge_2_arr.append(rouge_2) 
    rouge_L_arr.append(rouge_L) 
    rouge_be_arr.append(rouge_be) 

    i+=1

  top.set('bleu', str(reduce(lambda x, y: x + y,  bleu_arr) / len(bleu_arr)))
  top.set('rouge_1', str(reduce(lambda x, y: x + y,  rouge_1_arr) / len(rouge_1_arr)))
  top.set('rouge_2', str(reduce(lambda x, y: x + y,  rouge_2_arr) / len(rouge_2_arr)))
  top.set('rouge_L', str(reduce(lambda x, y: x + y,  rouge_L_arr) / len(rouge_L_arr)))
  top.set('rouge_be', str(reduce(lambda x, y: x + y, rouge_be_arr) / len(rouge_be_arr)))

  with open(default_path + "result.xml", "w+") as f:
    print(prettify(top), file=f)

#Run

In [0]:
zaksum(article,reference,summary,"drive/Colab Notebooks/Model 4 generator/")